# ***NEED TO CLARIFY WHAT THIS IS DOING***
This notebook ***(change all instances of notebook)*** loads all linguistic feature files calculated ***(in the previous scripts)***, and produces a single DataFrame containing the relevant variables for classification analysis. ***list the features***

In [23]:
# Importing necessary libraries
import pandas as pd
from scipy.stats import zscore

In [24]:
# Loading the linguistic features file
ling_features = pd.read_csv('linguistic_features/linguistic_features.csv', index_col = 0)

# Creating a DataFrame with the relevant features
ling_features_finalized = ling_features[
    ['Coherence', 
    'n_words', 
    'POS:NOUN', 
    'POS:PROPN', 
    'POS:PRON', 
    'POS:ADJ', 
    'POS:ADV', 
    'POS:CCONJ', 
    'POS:SCONJ', 
    'POS:DET', ''
    'open_closed_ratio', 
    'type_token_ratio', 
    'propositional_density', 
    'n_logical_operators', 
    'log10_freq_mean', 
    'semantic_diversity_mean', 
    'aoa_mean', 
    'concreteness_mean', 
    'semantic_thematic_distance', 
    'surprisal',
    'coref_local',
    'coref_global',
    'n_constituents',
    'n_noun_phrases',
    'n_verb_phrases',
    'n_sub_clauses',
    'n_prep_phrases',
    'mean_np_length',	
    'mean_vp_length']
]

display(ling_features_finalized)

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,...,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,4,210.0,0.141700,0.012146,0.165992,...,49.0,11.0,13.0,2.354430,4.632653
1,5,256.0,0.109272,0.033113,0.162252,...,68.0,12.0,23.0,2.516129,7.161765
2,5,758.0,0.139459,0.052973,0.103784,...,183.0,41.0,79.0,3.094595,7.650273
3,4,812.0,0.165787,0.024287,0.119324,...,172.0,17.0,72.0,3.010526,6.168605
4,5,155.0,0.177778,0.005556,0.150000,...,40.0,11.0,8.0,2.484375,6.775000
...,...,...,...,...,...,...,...,...,...,...,...
1534,3,206.0,0.165254,0.000000,0.144068,...,55.0,14.0,14.0,2.871429,9.181818
1535,4,96.0,0.111111,0.000000,0.138889,...,27.0,5.0,5.0,1.703704,6.481481
1536,3,101.0,0.214286,0.000000,0.107143,...,20.0,5.0,12.0,2.424242,7.450000
1537,5,131.0,0.138158,0.026316,0.151316,...,36.0,3.0,9.0,2.204082,5.694444


In [25]:
# Loading the local coherence file
coherence = pd.read_csv('linguistic_features/coherence_results.csv')

# Calculating the mean local coherence each transcript
grouped_df = coherence.groupby('pt', as_index=False).mean(numeric_only = True)

# Creating a DataFrame with local coherence scores
local_coherence = grouped_df[['lc']].rename(columns = {'lc': 'local_coherence'})

display(local_coherence)

,local_coherence
0,0.274387
1,0.215685
2,0.161755
3,0.199047
4,0.375034
...,...
1534,0.268190
1535,0.323834
1536,0.295389
1537,0.258032


In [26]:
# Loading the text description features file
text_description = pd.read_csv('linguistic_features/text_description_ordered.csv', index_col = 0).drop('text', axis = 1)

display(text_description)

,dependency_distance_mean,prop_adjacent_dependency_relation_mean,first_order_coherence,second_order_coherence
0,1.984713,0.442565,0.366634,0.316578
1,2.568674,0.377021,0.310618,0.329741
2,2.731996,0.413946,0.352635,0.331568
3,2.332504,0.423813,0.308754,0.263125
4,2.442240,0.426583,0.318063,0.399475
...,...,...,...,...
1534,2.708896,0.422875,0.377143,0.412284
1535,2.563026,0.405010,0.401308,0.352386
1536,2.669339,0.444015,0.319754,0.358499
1537,2.715251,0.364776,0.314997,0.391217


In [27]:
# Concatenating the various feature DataFrames into a large set
linguistic_feature_set = pd.concat([ling_features_finalized, local_coherence, text_description], axis = 1)

# Dropping any rows with NaN values to avoid issues in classification
linguistic_feature_set = linguistic_feature_set.dropna()

# Normalizing the number of constituents and phrasal types by the number of words in a transcript (number of high-level constituents per word)
linguistic_feature_set[['n_constituents', 'n_noun_phrases', 'n_verb_phrases', 'n_sub_clauses', 'n_prep_phrases']] = linguistic_feature_set[['n_constituents', 'n_noun_phrases', 'n_verb_phrases', 'n_sub_clauses', 'n_prep_phrases']].div(linguistic_feature_set['n_words'], axis = 0)

In [28]:
# Normalizing each linguistic feature using z-scores
coherence_df = linguistic_feature_set['Coherence']
linguistic_feature_set_norm = linguistic_feature_set.drop('Coherence', axis = 1)
linguistic_feature_set_norm = linguistic_feature_set_norm.apply(zscore)
linguistic_feature_set_final = pd.concat([coherence_df, linguistic_feature_set_norm], axis = 1)

In [29]:
display(linguistic_feature_set_final)

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,...,local_coherence,dependency_distance_mean,prop_adjacent_dependency_relation_mean,first_order_coherence,second_order_coherence
0,4,0.031521,0.007830,-0.282531,0.624773,...,-0.262331,-1.087732,0.369707,-0.390776,-0.751544
1,5,0.371692,-0.719002,0.410407,0.537476,...,-0.761177,0.514575,-1.412746,-0.927500,-0.630440
2,5,4.083993,-0.042397,1.066779,-0.827151,...,-1.219473,0.962705,-0.408579,-0.524902,-0.613629
3,4,4.483324,0.547678,0.118736,-0.464441,...,-0.902561,-0.133443,-0.140238,-0.945360,-1.243337
4,5,-0.375206,0.816435,-0.500332,0.251526,...,0.592963,0.167656,-0.064916,-0.856162,0.011139
...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.001940,0.535744,-0.683940,0.113069,...,-0.314994,0.899323,-0.165763,-0.290076,0.128986
1535,4,-0.811512,-0.677771,-0.683940,-0.007805,...,0.157871,0.499077,-0.651595,-0.058541,-0.422100
1536,3,-0.774537,1.634691,-0.683940,-0.748751,...,-0.083855,0.790783,0.409152,-0.839955,-0.365860
1537,5,-0.552686,-0.071569,0.185778,0.282236,...,-0.401311,0.916759,-1.745756,-0.885538,-0.064838


In [30]:
# Saving the final linguistic feature set
linguistic_feature_set_final.to_csv('linguistic_features/linguistic_feature_set.csv')